In [12]:
MARKET = 'SNP'

In [13]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [14]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [15]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [16]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [17]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_fresh.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_symlots.pkl', 'df_und_margins.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl', 'qopts.pkl'],
      dtype='<U18')

In [40]:
# imports
from ib_insync import LimitOrder
from support import get_openorders

In [46]:
# ... check open ordes and remove them
df_openords = get_openorders(MARKET)
df1 = df_fresh[~df_fresh.conId.isin(df_openords.conId)].reset_index(drop=True)

In [47]:
# ... build the SELL orders
contracts = df1.contract.to_list()
orders = [LimitOrder(action='SELL', totalQuantity=abs(int(q)), lmtPrice=p) 
                        for q, p in zip(df1.qty, df1.expPrice)]

cos = [(c, o) for c, o in zip(contracts, orders)]

# Place the trades

In [21]:
# prepare the best to-dos

# ..sort by the rom
df = df_fresh.sort_values('rom', ascending = False)

# ... expected rom and sdMult mask
m = (df.rom > ibp.MINEXPROM) & np.where(df.right == 'P', df.sdMult > ibp.PUTSTDMULT, df.sdMult > ibp.CALLSTDMULT)

# ... right mask
r = (df.strike > df.undPrice) & (df.right == 'C') | (df.strike < df.undPrice) & (df.right == 'P')
df1 = df[m & r].sort_values(['right', 'strike'], ascending=[True, False])


In [39]:
cols = ['symbol', 'expiry', 'strike', 'right', 'dte', 'price', 'margin', 'expPrice', 'rom', 'sdMult', 'qty']
df1[cols]

,symbol,expiry,strike,right,dte,price,margin,expPrice,rom,sdMult,qty
1,DJX,20201125,308.0,C,2,2.93,1669.33,2.93,31.842764,3.500563,1
3,MCHP,20201127,165.0,C,4,0.46,294.51,0.50,13.761147,5.114031,1
16,MCHP,20201231,165.0,C,38,1.86,590.76,1.86,2.998414,2.321833,1
4,MCHP,20201127,160.0,C,4,0.53,447.44,0.53,10.485306,4.589213,1
13,CXO,20201127,70.0,C,4,0.10,209.16,0.50,3.409004,4.550722,1
17,CXO,20201127,66.0,C,4,0.15,407.02,0.50,2.872770,2.409910,1
9,HPE,20201211,17.0,C,18,0.10,41.25,0.50,3.841213,6.488231,1
10,HPE,20201211,16.5,C,18,0.10,41.25,0.50,3.841205,5.989136,1
11,HPE,20201211,15.5,C,18,0.10,41.25,0.50,3.841190,4.990947,1
5,HPE,20201211,15.0,C,18,0.15,41.25,0.50,6.299095,4.491852,1


In [35]:
from support import watchlist
watchlist?

Signature: watchlist(MARKET: str, symbols: list, FILE_NAME='watchlist.csv') -> pandas.core.frame.DataFrame
Docstring: <no docstring>
File:      c:\users\user\documents\business\projects\learn\ib\support.py
Type:      function


In [36]:
watchlist('SNP', df1.symbol.to_list())

,symbol,secType,primaryExchange
0,DJX,IND,CBOE
34,AZO,STK,SMART/AMEX
74,CXO,STK,SMART/AMEX
88,EA,STK,SMART/AMEX
118,HFC,STK,SMART/AMEX
121,HPE,STK,SMART/AMEX
158,MCHP,STK,SMART/AMEX
201,REGN,STK,SMART/AMEX
